In [ ]:
## install and upgrate necessary packages
import sys
!{sys.executable} -m pip install drive/MyDrive/pyds &> /dev/null
!{sys.executable} -m pip install --upgrade imblearn &> /dev/null

## add mature code to PATH
sys.path.append('/content/drive/MyDrive')

In [ ]:
## Import essential packages for IdAuth
%matplotlib inline

import os
import json
import copy
from collections import Counter
import numpy as np
import pandas as pd
import imblearn.over_sampling

import tensorflow as tf

from idauth.datasets.hmog import HMOGProvider
from idauth.authenticators.touch import TouchAuth
from idauth.authenticators.gait import DeepGaitExtractionModel, DeepGaitAuthenticator, DeepGaitModel
from idauth.utils.sliding_window import SlidingWindow
from idauth.utils.helpers import balance_dataset, sparse_label_converter
from idauth.utils.metrics import eer
from idauth.simulator.imposter import ImposterProvider
from idauth.simulator.timeline import Timeline
from idauth.storage import BiometricStorage, SCHEME_TIME_SEQ_10

from sklearn.metrics import roc_curve, roc_auc_score, precision_score, recall_score, f1_score,auc,roc_curve, accuracy_score
import matplotlib.pyplot as plt

import logging

In [ ]:
## GLOBAL CONFIGURATION
LOG_OUTPUT_PATH = ""
LOG_FILENAME = 'c07.log'
logging.basicConfig(format='%(asctime)-15s %(message)s',
                    filename=os.path.join(LOG_OUTPUT_PATH, LOG_FILENAME),
                    level=logging.DEBUG)
logger = logging.getLogger('simulator')

SIMULATION_RESULT_DIR = "simulation"
GAME_DETAILS_ENABELD = False # to show training and testing sessions
VISUABLIZATION_ENABLED = False # show a timeline figure to visualize decisions

In [ ]:
def simple_merge_acc_gyro(acc_df, gyro_df, step=2):
    size = min(acc_df.shape[0], gyro_df.shape[0])
    acc = acc_df.iloc[:size][['x', 'y', 'z']].to_numpy()
    gyro = gyro_df.iloc[:size][['x', 'y', 'z']].to_numpy()
    merged = np.concatenate((acc, gyro), axis=1)
    return merged[::step]

def touch_df_to_features(df):
    raw = df[['swipe_id', 'time', 'x', 'y', 'pressure', 'area', 'orientation',
              'activity']].values
    features = TouchAuth().extract_features(raw) # to be staticmethod
    return features

def get_touch_features_by_user_session(provider: HMOGProvider,
                                       user_session_settings):
    """
    Obtain a dictionary like:"
      result['<user_name>'][<session_id>]: feature list
    """
    result = {}
    for user, sessions in user_session_settings.items():
        result[user] = {}
        for sess in sessions:
            features = touch_df_to_features(provider.data[user][sess]['touch'])
            result[user][sess] = features
    return result

def get_gait_features_by_user_session(provider: HMOGProvider,
                                      user_session_settings):
    result = {}
    for user, sessions in user_session_settings.items():
        result[user] = {}
        for sess in sessions:
            tmp_data = simple_merge_acc_gyro(hmog.data[user][sess]['acc'],
                                         hmog.data[user][sess]['gyro'])
            features = DeepGaitAuthenticator().extract_features(tmp_data)
            result[user][sess] = features
    return result

def get_gait_features(provider, user, session):
    tmp_data = simple_merge_acc_gyro(provider.data[user][session]['acc'],
                                     provider.data[user][session]['gyro'])
    features = DeepGaitAuthenticator().extract_features(tmp_data)
    return features

def get_touch_features(provider, user, session):
    features = touch_df_to_features(provider.data[user][session]['touch'])
    return features

def get_features(provider, auth_type, user, session):
    if auth_type == 'gait':
        return get_gait_features(provider, user, session)
    elif auth_type == 'touch':
        return get_touch_features(provider, user, session)

###
## Timeline operations
###

def get_events_for_user_session(user, session, new_basetime):
    logger.info("generate events for: " + str(user) + ", " + str(session))
    base_time = int(min(hmog.data[user][session]['acc'].iloc[0]['time'],
                         hmog.data[user][session]['gyro'].iloc[0]['time']))
    # e.g. new_basetime starts from 0, base_time = 1000, t0 = 1003
    # offset = - base_time + new_basetime
    # offset = - 1000 + 0
    # new_timestamp = t0 + offset = 1003 - 1000 = 3
    offset = new_basetime - base_time
    acc_events = get_acc_events_for_user_session(user, session, offset)
    gyro_events = get_gyro_events_for_user_session(user, session, offset)
    touch_events = get_touch_events_for_user_session(user, session, offset)
    merged_events = sorted(acc_events + gyro_events + touch_events)
    return merged_events

def get_events_for_user_session_with_limit(user, session, new_basetime, limit):
    logger.info("generate events for: " + str(user) + ", " + str(session))
    base_time = int(min(hmog.data[user][session]['acc'].iloc[0]['time'],
                        hmog.data[user][session]['gyro'].iloc[0]['time']))
    end_time = base_time + limit
    # e.g. new_basetime starts from 0, base_time = 1000, t0 = 1003
    # offset = - base_time + new_basetime
    # offset = - 1000 + 0
    # new_timestamp = t0 + offset = 1003 - 1000 = 3
    offset = new_basetime - base_time
    acc_events = get_acc_events_for_user_session(user, session, offset, end_time)
    gyro_events = get_gyro_events_for_user_session(user, session, offset, end_time)
    touch_events = get_touch_events_for_user_session(user, session, offset, end_time)
    merged_events = sorted(acc_events + gyro_events + touch_events)
    return merged_events


def get_acc_events_for_user_session(user, session, offset, end_time=None, start_time=None):
    df = hmog.data[user][session]['acc'].iloc[::2]
    result = []
    for index, row in df.iterrows():
        if start_time is not None and int(row['time']) < start_time:
            continue
        if end_time is not None and int(row['time']) > end_time:
            break
        result.append((int(row['time']) + offset, 'acc',
                       (row['x'], row['y'], row['z'])))
    return result

def get_gyro_events_for_user_session(user, session, offset, end_time=None, start_time=None):
    df = hmog.data[user][session]['gyro'].iloc[::2]
    result = []
    for index, row in df.iterrows():
        if start_time is not None and int(row['time']) < start_time:
            continue
        if end_time is not None and int(row['time']) > end_time:
            break
        result.append((int(row['time']) + offset, 'gyro',
                       (row['x'], row['y'], row['z'])))
    return result

def get_touch_events_for_user_session(user, session, offset, end_time=None, start_time=None):
    df = hmog.data[user][session]['touch']
    result = []
    for index, row in df.iterrows():
        if start_time is not None and int(row['time']) < start_time:
            continue
        if end_time is not None and int(row['time']) > end_time:
            break
        result.append((int(row['time']) + offset, 'touch',
                       (row['swipe_id'], row['time'] + offset, row['x'], row['y'],
                        row['pressure'], row['area'], row['orientation'], row['activity'])))
    return result

In [ ]:
from scipy.spatial import distance

class BoundedStorage:
    # in bounded storage, we do not have cross session splitting
    # however, all data is kept in a sequence

    IMPOSTER_NAME = -1

    @staticmethod
    def merge_two(a, b):
        if len(a) == 0:
            return np.array(b)
        if len(b) == 0:
            return np.array(a)
        return np.concatenate([a, b], axis=0)

    def set_max_space(self, auth_type, size):
        self.max_space[auth_type] = size

    def construct(self, auth, data, func=None):
        """
        select the most important features for training
        should have a function to calculate the "weight" of each
        feature vector
        or follow certain criteria to keep the samples
        new_data: new incoming data for auth type, a dict of <user>: <features>
        """
        ranked_list = []
        result = []

        if auth == 'gait':
            fea_data = func[auth](data)
        else:
            fea_data = data

        mean = np.average(fea_data, axis=0)
        for n in range(self.max_space[auth]):
            index = self.find_best_feature(auth, fea_data, ranked_list, mean)
            if index < 0:
                break

            ranked_list.append(fea_data[index])
            result.append(data[index])
            fea_data = np.delete(fea_data, index, 0)

        print("construct", auth, len(ranked_list))
        return result

    def find_best_feature(self, auth, data):
        pass


    def add_user(self, user):
        self.users.append(user)
        self.data[user] = {
            auth: [] for auth in self.auth_types
        }
        self.cache[user] = {
            auth: [] for auth in self.auth_types
        }

    def initialize_storage(self):
        for user in self.users:
            self.data[user] = {
                auth: [] for auth in self.auth_types
            }
            self.cache[user] = {
                auth: [] for auth in self.auth_types
            }

    def add_data(self, user, auth, data):
        """
        what happens when we add a new session's data
        use the new data and old data to update a model?
        no
        our current solution is to add data and run reduce without retraining
        the whole model

        """
        self.cache[user][auth] = BoundedStorage.merge_two(self.cache[user][auth],
                                                      data)

    def update_storage(self, func=None):
        for auth in self.auth_types:
            for user in self.users:
                cached_data = self.cache[user][auth]
                stored_data = self.data[user][auth]


                if len(cached_data) + len(stored_data) <= self.max_space[auth]:
                    self.data[user][auth] = BoundedStorage.merge_two(stored_data, cached_data)
                else:
                    self.data[user][auth] = self.construct(auth,
                                                           BoundedStorage.merge_two(stored_data, cached_data),
                                                           func)

        for user in self.users:

            self.cache[user] = {
                auth: [] for auth in self.auth_types
            }
        print("update storage size finished")

    def split_scheme(self, user, auth):
        pass

    def get_train_test_sets(self, auth):
        result = {
            'train': {},
            'test': {}
        }

        for user in self.users:
            train_, test_ = self.split_scheme(user, auth)
            result['train'][user] = train_
            result['test'][user] = test_

        result['train'][self.IMPOSTER_NAME] = self.imposters[auth]['train']
        result['test'][self.IMPOSTER_NAME] = self.imposters[auth]['test']

        return result

    def __init__(self, users=None, auth_types=None, imposter_set=None,
                 max_space=None):
        self.users = [] if users is None else users
        self.auth_types = auth_types
        self.data = dict()
        self.cache = dict()
        self.imposters = imposter_set
        self.max_space = max_space
        self.initialize_storage()


class UnBoundedStorage(BoundedStorage):
    def construct(self, auth, data, func):
        return data

    def split_scheme(self, user, auth):
        # print(auth, np.array(self.data[user][auth]).shape, np.array(self.cache[user][auth]).shape)
        all_data = BoundedStorage.merge_two(self.data[user][auth], self.cache[user][auth])
        test_size = int(all_data.shape[0] * 0.1)
        if test_size == 0:
            test_size = 1
        test_set = all_data[-test_size:]
        train_set = all_data[:-test_size]
        print(user, auth, train_set.shape, test_set.shape)
        return train_set, test_set

class MeanDistanceStorage(BoundedStorage):
    def find_best_feature(self, auth, data, sel_data, mean):
        min_index = -1
        min_dist = np.inf
        sum_sel = np.sum(sel_data, axis=0)
        # print(mean.shape)
        for i, item in enumerate(data):
            new_average = (item + sum_sel) / (len(sel_data) + 1)
            assert new_average.shape == mean.shape
            dist = distance.euclidean(new_average, mean)
            if min_index == -1 or dist < min_dist:
                min_dist = dist
                min_index = i
        return min_index


    def split_scheme(self, user, auth):
        # print(auth, np.array(self.data[user][auth]).shape, np.array(self.cache[user][auth]).shape)
        all_data = BoundedStorage.merge_two(self.data[user][auth], self.cache[user][auth])
        test_size = int(all_data.shape[0] * 0.1)
        if test_size == 0:
            test_size = 1
        test_set = all_data[-test_size:]
        train_set = all_data[:-test_size]
        print(user, auth, train_set.shape, test_set.shape)
        return train_set, test_set


class RandomStorage(BoundedStorage):
    def find_best_feature(self, auth, data, sel_data, mean):
        return np.random.randint(len(data))

    def split_scheme(self, user, auth):
        # print(auth, np.array(self.data[user][auth]).shape, np.array(self.cache[user][auth]).shape)
        all_data = BoundedStorage.merge_two(self.data[user][auth], self.cache[user][auth])
        test_size = int(all_data.shape[0] * 0.1)
        if test_size == 0:
            test_size = 1
        test_set = all_data[-test_size:]
        train_set = all_data[:-test_size]
        print(user, auth, train_set.shape, test_set.shape)
        return train_set, test_set



storage = MeanDistanceStorage([1,2], ['touch', 'gait'],
                    {
                        'touch': {'train': [[1,1,1], [2,2,2]],
                                   'test': [[1,2,1], [2,1,2]]},
                         'gait': {'train':  [[0,0,1,1,1], [2,3,4,1,1]],
                                  'test': [[0,0,2,1,1], [1,2,5,2,1]]}
                    },
                    {
                        'touch': 5,
                        'gait': 3
                    }
                    )




def test_storage_add(user, auth, size, num):
    fea_v = np.random.randn(num, size) * 2 + 5
    print(fea_v.tolist())
    storage.add_data(user, auth, fea_v.tolist())


# test_storage_add(1, 'touch', 3, 5)
# test_storage_add(2, 'touch', 3, 5)
# test_storage_add(1, 'gait', 5, 5)
# test_storage_add(2, 'gait', 5, 5)
# storage.gen_train_test_sets('touch')
# storage.update_storage()

# test_storage_add(1, 'touch', 3, 5)
# test_storage_add(2, 'touch', 3, 5)
# test_storage_add(1, 'gait', 5, 5)
# test_storage_add(2, 'gait', 5, 5)
# storage.gen_train_test_sets('touch')
# storage.update_storage()

# test_storage_add(1, 'touch', 3, 5)
# test_storage_add(2, 'touch', 3, 5)
# test_storage_add(1, 'gait', 5, 5)
# test_storage_add(2, 'gait', 5, 5)
# storage.gen_train_test_sets('touch')
# storage.update_storage()

# print(storage.data)


In [ ]:
a = [1,2,3,4]
b = a
b.pop(0)
print(a)

[2, 3, 4]


In [ ]:
## DS FUNC
import pyds as ds
import functools

def ds_fusion_2(scores, uncertainties):
    mass_groups = [ds.MassFunction({'a': s * (1 - u), 'b': (1 - s) * (1 - u),
                                    'ab': u})
                   for s, u in zip(scores, uncertainties)]
    combined_mass = functools.reduce(lambda a, b: a & b, mass_groups)
    return combined_mass['a']


# =======================================
#  Scoring Strategy
# =======================================

def gait_scoring(buffer, t0):
    raw = [data for auth_type, data, t in buffer
           if t == t0 and auth_type == 'gait']
    return raw

def touch_scoring(buffer, t0):
    raw = [data for auth_type, data, t in buffer
           if t == t0 and auth_type == 'touch']
    return raw

def plain_average_scoring(buffer, t0, time_interval=20000):
    target_index = len(buffer) - 1
    for i in range(len(buffer) - 1, 0, -1):
        if np.abs(t0 - buffer[i][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    # print(sel_buffer)
    auth_types, results, times = zip(*sel_buffer)
    return [np.array(np.average(results, axis=0))]


def weighted_average_eer_scoring(buffer, t0, time_interval=20000, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer) - 1, 0, -1):
        if np.abs(t0 - buffer[i][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    # print(sel_buffer)

    weighted_scores = [[r * max(0, 1 - 2*strategy[auth_type]['eer'][i])
                        for i, r in enumerate(result)]
                        for auth_type, result, time in sel_buffer]

    return [np.array(np.average(weighted_scores, axis=0))]


def weighted_average_auc_scoring(buffer, t0, time_interval=20000, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer) - 1, 0, -1):
        if np.abs(t0 - buffer[i][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    weighted_scores = [[r * min(1,  strategy[auth_type]['auc'][i])
                        for i, r in enumerate(result)]
                        for auth_type, result, time in sel_buffer]

    return [np.array(np.average(weighted_scores, axis=0))]

def deer_ds_scoring(buffer, t0, time_interval=20000, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer), 0, -1):
        if np.abs(t0 - buffer[i - 1][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    auth_types, results, times = zip(*sel_buffer)

    modalities = dict()
    for auth_type, result, time in sel_buffer:
        if auth_type not in modalities:
            modalities[auth_type] = [result]
        else:
            modalities[auth_type].append(result)

    if len(modalities.keys()) < 2:
        return [np.average(modalities[list(modalities.keys())[0]], axis=0)]

    result_gait = np.average(modalities['gait'], axis=0)
    result_touch = np.average(modalities['touch'], axis=0)

    final_result = []
    for i in range(result_gait.shape[0]):
        scores = [result_gait[i], result_touch[i]]
        uncertainty = [min(1, 2*strategy['gait']['eer'][i]),
                       min(1, 2*strategy['touch']['eer'][i])]

        result = ds_fusion_2(scores, uncertainty)
        # print(scores, uncertainty, result)
        final_result.append(result)

    return [np.array(final_result)]


def redeer_ds_scoring(buffer, t0, time_interval=20000, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer), 0, -1):
        if np.abs(t0 - buffer[i - 1][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    auth_types, results, times = zip(*sel_buffer)

    modalities = dict()
    for auth_type, result, time in sel_buffer:
        if auth_type not in modalities:
            modalities[auth_type] = [result]
        else:
            modalities[auth_type].append(result)

    if len(modalities.keys()) < 2:
        return [np.average(modalities[list(modalities.keys())[0]], axis=0)]

    result_gait = np.average(modalities['gait'], axis=0)
    result_touch = np.average(modalities['touch'], axis=0)

    final_result = []
    for i in range(result_gait.shape[0]):
        scores = [result_gait[i], result_touch[i]]
        uncertainty = [max(0, 1 - 2 * strategy['gait']['eer'][i]),
                       max(0, 1 - 2 * strategy['touch']['eer'][i])]

        result = ds_fusion_2(scores, uncertainty)
        # print(scores, uncertainty, result)
        final_result.append(result)

    return [np.array(final_result)]

def auc_ds_scoring(buffer, t0, time_interval=20000, strategy=None):
    target_index = len(buffer) - 1
    for i in range(len(buffer), 0, -1):
        if np.abs(t0 - buffer[i - 1][2]) > time_interval:
            break
        else:
            target_index = i - 1

    sel_buffer = buffer[target_index:]
    auth_types, results, times = zip(*sel_buffer)

    modalities = dict()
    for auth_type, result, time in sel_buffer:
        if auth_type not in modalities:
            modalities[auth_type] = [result]
        else:
            modalities[auth_type].append(result)

    if len(modalities.keys()) < 2:
        return [np.average(modalities[list(modalities.keys())[0]], axis=0)]

    result_gait = np.average(modalities['gait'], axis=0)
    result_touch = np.average(modalities['touch'], axis=0)

    final_result = []
    for i in range(result_gait.shape[0]):
        scores = [result_gait[i], result_touch[i]]
        uncertainty = [max(0, 1 - strategy['gait']['auc'][i]),
                       max(0, 1 - strategy['touch']['auc'][i])]

        result = ds_fusion_2(scores, uncertainty)
        # print(scores, uncertainty, result)
        final_result.append(result)

    return [np.array(final_result)]


SUPPORTED_SCORER = {
    'simple_average': plain_average_scoring,
    'weighted_average_eer': weighted_average_eer_scoring,
    'weighted_average_auc': weighted_average_auc_scoring,
    # 'dempster_shafer': dempster_shafer_score,
    # 'redeer_dempster_shafer': redeer_ds_scoring,
    'deer_dempster_shafer': deer_ds_scoring,
    'auc_dempster_shafer': auc_ds_scoring,
    # 'fixed_dempster_shafer': fixed_dempster_shafer_score
}


In [ ]:
GAIT_NO_GAITS_DETECTED = -101
GAIT_GAITS_DETECTED = 101
GAIT_INSUFFICIENT_STEPS = -100

TOUCH_NO_SWIPE_DETECTED = -201
TOUCH_SWIPE_DETECTED = 201

DECISION_INSUFFICIENT_DATA = -1

EVENT_DEVICE_SHARING = "device sharing"
DISCARD_CLIP_LENGTH = 60000

### comment types
COMMENT_FALSE_REJECTION = "false_rejection"
COMMENT_FALSE_ACCEPTANCE = "false_acceptance"
COMMENT_FALSE_MISCLASSIFICATION = "false_classification"
COMMENT_CONFIRM_USER_CHANGE = "confirm_user_change"
COMMENT_DENY_USER_CHANGE = "deny_user_change"

### special events
EVENT_SESSION_START = "session_start"
EVENT_SESSION_END = "session_end"
EVENT_ACCESS_GRANT = "access_grant"
EVENT_TRAIN_MODELS = "train_models"
EVENT_USER_CHANGE = "user_change" # user change signal is only for ground truth
EVENT_ADD_USER = "add_user"

SPECIAL_EVENTS = [
    EVENT_SESSION_START,
    EVENT_SESSION_END,
    EVENT_ACCESS_GRANT,
    EVENT_TRAIN_MODELS,
    EVENT_USER_CHANGE
]

### system states
STATE_ENROLLMENT = "state_enrollment"
STATE_NORMAL = "state_normal"
STATE_SHARING = "state_sharing"

SYSTEM_STATES = [
    STATE_ENROLLMENT,
    STATE_NORMAL,
    STATE_SHARING
]

### user states
USER_UNKNOWN = "Unknown"
USER_STRANGER = "Stranger"

### reaction types
REACTION_NONE = 0
REACTION_REJECT = -1
REACTION_USER_CHANGE = 9
REACTION_ACCEPT = 1

### default player configuration
DEFAULT_CONFIGURATION = {
    'decision_maker': 'deer_dempster_shafer',
    'ext_sliding_window': (7, 4),
    'model_update': True,
    'accept_user_change_signal': False,
    'storage': 'mean_distance'
}


class TestPlayer:

    def train_auth(self, auth_type):
        auth = self.auth[auth_type]
        settings = self.gen_train_test_set(auth_type)
        reports = auth.train_model_with_eval((settings['train_label'],
                                              settings['train_data']),
                                             (settings['test_label'],
                                              settings['test_data']))

        return reports

    def train_all(self):
        for name, auth in self.auth.items():
            self.auth_report[name] = self.train_auth(name)
            print(name)
            print(self.auth_report[name]['report'])

        self.storage.update_storage({
            'gait': self.auth['gait'].get_final_layer
        })

    def gen_train_test_set(self, auth_type):
        train_data_set = []
        train_label_set = []
        test_data_set = []
        test_label_set = []

        train_test_dict = self.storage.get_train_test_sets(auth_type)
        for key, tt in train_test_dict['train'].items():
            label = self.get_user_label(key)
            train_data_set.append(tt)
            train_label_set.append(np.ones(tt.shape[0], dtype=int) * label)
        for key, tt in train_test_dict['test'].items():
            label = self.get_user_label(key)
            test_data_set.append(tt)
            test_label_set.append(np.ones(tt.shape[0], dtype=int) * label)

        # # sample test dataset
        train_label, train_data = balance_dataset(np.concatenate(train_label_set),
                                                  np.concatenate(train_data_set),
                                                  int(self.storage.max_space[auth_type] / 5),
                                                  1,
                                                  0)
        test_label, test_data = balance_dataset(np.concatenate(test_label_set),
                                                np.concatenate(test_data_set),
                                                int(self.storage.max_space[auth_type] / 5),
                                                1,
                                                0)

        # print("train_data:", train_label.shape[0], "test_data", test_label.shape[0])

        return {
            'train_data': train_data, # np.concatenate(train_data_set),
            'train_label': train_label, # np.concatenate(train_label_set),
            'test_data': test_data,
            'test_label': test_label
        }

    def feedback(self, comment):
        """
        feedback function receives user's feedback or other signal
        one representative example is: user correct misclassification
        comment format:
        (COMMENT_TYPE, DETAILS)
        """
        com_type, com = comment
        if com_type == COMMENT_FALSE_REJECTION:
            self.current_user = com
            self.reset_buffers()
        elif com_type == COMMENT_FALSE_MISCLASSIFICATION:
            self.current_user = com
        elif com_type == COMMENT_CONFIRM_USER_CHANGE:
            # only after getting confirmation then update cached data to storage
            print("system gets feedback, CONFIRMED!")
            if self.flag_store_history:
                self.update_cache_to_storage(self.previous_user, crop=DISCARD_CLIP_LENGTH)
        elif com_type == COMMENT_DENY_USER_CHANGE:
            print("system gets feedback, DENIED!", com)
            self.current_user = com
            self.reset_buffers()


    def __handle_special_events(self, event):
        if event == EVENT_SESSION_START:
            # handle a session start
            ## TODO: tasks that need to be done at session start
            pass

        elif event == EVENT_SESSION_END:
            # handle a session end
            if self.status == STATE_SHARING:
                self.status = STATE_NORMAL

            # assign allocation retrain
            # print("session end, train model")
            if self.flag_store_history:
                self.update_cache_to_storage(self.current_user)
            # self.train_all()

            # reset current user to unknown
            self.current_user = USER_UNKNOWN
            self.previous_user = USER_UNKNOWN

            # reset buffer and decisions
            self.buffer = {
                'acc': [],
                'gyro': [],
                'touch': []
            }
            self.decision_sw.reset()
            self.last_update = {
                'acc': -1,
                'gyro': -1,
                'touch': -1
            }

            self.detect_ctr = {
                'gait': 0
            }

            self.cur_swipe = -1
            self.decision_buffer = []
            self.another_buffer = []


        elif event == EVENT_ACCESS_GRANT:
            # handle a device sharing
            self.set_status(STATE_SHARING)

        elif event == EVENT_TRAIN_MODELS and self.flag_store_history:
            self.train_all()

        elif event == EVENT_USER_CHANGE and self.accept_signal:
            # handle a session end
            if self.status == STATE_SHARING:
                self.status = STATE_NORMAL

            # assign allocation retrain
            # print("session end, train model")
            if self.flag_store_history:
                self.update_cache_to_storage(self.current_user)
            # self.train_all()

            # reset current user to unknown
            self.previous_user = self.current_user
            self.current_user = USER_UNKNOWN

            # reset buffer and decisions
            self.buffer = {
                'acc': [],
                'gyro': [],
                'touch': []
            }
            self.decision_sw.reset()
            self.last_update = {
                'acc': -1,
                'gyro': -1,
                'touch': -1
            }

            self.detect_ctr = {
                'gait': 0
            }

            self.cur_swipe = -1
            self.decision_buffer = []
            self.another_buffer = []

    def reset_buffers(self):
        # reset buffer and decisions
        self.buffer = {
            'acc': [],
            'gyro': [],
            'touch': []
        }
        self.decision_sw.reset()
        self.last_update = {
            'acc': -1,
            'gyro': -1,
            'touch': -1
        }

        self.detect_ctr = {
            'gait': 0
        }

        self.cur_swipe = -1
        self.decision_buffer = []
        self.another_buffer = []

    def dispatch(self, event):
        time, sensor, data = event
        self.cur_time = time

        if sensor in SPECIAL_EVENTS:
            self.__handle_special_events(sensor)
            return REACTION_NONE

        self.buffer[sensor].append(data)

        reaction = REACTION_NONE # default reaction is nothing

        ## gait checker
        if sensor == 'acc':
            if len(self.buffer['acc']) >= 1024 \
              and len(self.buffer['gyro']) >= 1024 \
              and self.detect_ctr['gait'] >= 512:
                result = self.gait_detection()
                if result['code'] == GAIT_GAITS_DETECTED:
                    decision = self.aggregate("gait", result['result'], time)
                    if decision >= 0:
                        reaction = self.react_to_decision(time, decision)
                self.detect_ctr['gait'] = 0
            else:
                self.detect_ctr['gait'] += 1


        if sensor == 'touch':
            if self.cur_swipe == -1:
                self.cur_swipe = data[0]
            elif data[0] != self.cur_swipe:
                result = self.touch_detection()
                self.cur_swipe = data[0]
                if result['code'] == TOUCH_SWIPE_DETECTED:
                    decision = self.aggregate("touch", result['result'], time)
                    if decision >= 0:
                        reaction = self.react_to_decision(time, decision)
        return reaction

    def is_valid_switch(self, time):
        if self.last_switch_time < 0 or time - self.last_switch_time > self.QUESTIONABLE_SWITCH_INTERVAL:
            # valid switch
            return True
        else:
            return False

    def react_to_decision(self, time, decision):
        if decision < 0:
            # invalid decision
            return REACTION_NONE

        if decision == 0:
            self.current_user = USER_STRANGER
            if self.status == STATE_SHARING:
                return REACTION_NONE

            return REACTION_REJECT

        if self.current_user == USER_UNKNOWN:
            self.current_user = self.user_profile[decision - 1]
            return REACTION_ACCEPT
        else:
            if self.current_user != self.user_profile[decision - 1]:
                # is_valid = self.is_valid_switch(time)
                # if is_valid:
                self.previous_user = self.current_user
                self.current_user = self.user_profile[decision - 1]
                print("[test] system capture user change", self.cur_time, time, self.previous_user, self.current_user)
                return REACTION_USER_CHANGE
            else:
                return REACTION_ACCEPT

    def cache_auth_data(self, auth_type, time, features):
        self.biometric_cache[auth_type].append((time, features))

    def update_cache_to_storage(self, true_user, crop=0):
        if true_user not in self.user_profile:
            print(true_user, self.user_profile)
            print("current session does not belong to a valid user, discarded")
            self.clear_cache()
            return

        for auth_type, cached_data in self.biometric_cache.items():
            if len(cached_data) == 0:
                continue

            features = [feature for time, feature in cached_data
                        if time + crop <= self.cur_time]
            print("cached", len(features), 'features')
            self.storage.add_data(true_user,
                                          auth_type,
                                          np.array(features))
        self.clear_cache()

    def clear_cache(self):
        self.biometric_cache = {
            'gait': [],
            'touch': []
        }

    def gait_detection(self):
        dga = self.auth['gait']
        acc = np.array(self.buffer['acc'][-1024:])
        gyro = np.array(self.buffer['gyro'][-1024:])
        merged = np.concatenate([acc, gyro], axis=1)
        processed = np.array([merged]).reshape(1, 1024, 6, 1)
        result = dge.predict(processed.transpose(0, 2, 1, 3))[0]
        gaits = []
        st = 0
        flag = False
        tolerant_ctr = 0
        max_tolerance = 5
        for i in range(result.shape[0]):
            if result[i] > 0.5:
                if not flag:
                    st = i
                    flag = True
                tolerant_ctr = 0
            elif flag:
                if tolerant_ctr >= max_tolerance:
                    if i - st > 128:
                        gaits.append((st, i))
                        flag = False
                    else:
                        tolerant_ctr = 0
                        flag = False
                else:
                    tolerant_ctr += 1
        if flag and result.shape[0] - 1 - st > 128:
            gaits.append((st, result.shape[0] - 1))

        if len(gaits) == 0:
            return {
                'code': GAIT_NO_GAITS_DETECTED,
                'result': None
            }

        auth_result = None
        for gait in gaits:
            features = dga.extract_features(merged[gait[0]: gait[1] + 1])
            if features is None:
                continue
            else:
                r = dga.authenticate(features)
                for feature, result in zip(features, r):
                    self.cache_auth_data('gait', self.cur_time, feature)

                auth_result = r if auth_result is None else np.concatenate((auth_result, r), axis=0)
        if auth_result is None or len(auth_result) == 0:
            return {
                'code': GAIT_NO_GAITS_DETECTED,
                'result': None
            }
        else:

            return {
                'code': GAIT_GAITS_DETECTED,
                'result': auth_result.tolist()
            }


    def touch_detection(self):
        toa = self.auth['touch']
        features = toa.extract_features(self.buffer['touch'])
        if len(features) == 0:
            return {
                'code': TOUCH_NO_SWIPE_DETECTED,
                'result': None
            }
        else:
            result = toa.authenticate([features[-1]])
            self.cache_auth_data('touch', self.cur_time, features[-1])
            return {
                'code': TOUCH_SWIPE_DETECTED,
                'result': result
            }

    def aggregate(self, auth_type, raw_result, time):
        self.decision_buffer += [(auth_type, i, time) for i in raw_result]

       ### record all decision scores
        if auth_type == 'gait':
            result = gait_scoring(self.decision_buffer, time)
            self.decision_scores[auth_type].append((time, np.average(result, axis=0)))

        if auth_type == 'touch':
            result = touch_scoring(self.decision_buffer, time)
            self.decision_scores[auth_type] += [(time, r) for r in result]

        for dem_name, dem_maker in SUPPORTED_SCORER.items():
            if dem_name == 'simple_voting':
                result = dem_maker(self.another_buffer, time)
                self.decision_scores[dem_name] += [(time, r) for r in result]
            elif 'weighted_average' in dem_name:
                result = dem_maker(self.decision_buffer, time,
                                   strategy=self.auth_report)
                self.decision_scores[dem_name] += [(time, r) for r in result]
            elif 'dempster_shafer' in dem_name:
                result = dem_maker(self.decision_buffer, time,
                                   strategy=self.auth_report)
                self.decision_scores[dem_name] += [(time, r) for r in result]
            else:
                result = dem_maker(self.decision_buffer, time)
                self.decision_scores[dem_name] += [(time, r) for r in result]

        result = None
        primary_dmaker = SUPPORTED_SCORER[self.primary_dm]
        if 'weighted_average' in self.primary_dm:
            result = primary_dmaker(self.decision_buffer, time,
                                    strategy=self.auth_report)
        elif 'dempster_shafer' in self.primary_dm:
            result = primary_dmaker(self.decision_buffer, time,
                                    strategy=self.auth_report)
        else:
            result = primary_dmaker(self.decision_buffer, time)

        dec = SlidingWindow.RESULT_DATA_INSUFFICIENT if result is None else \
          self.decision_sw.add(np.argmax(result[0]))
        # if result is not None:
        #     print("my answer:", dec, result[0])
        return dec

    # =====================================
    # System status management functions
    # =====================================
    def set_status(self, state):
        assert state in SYSTEM_STATES
        self.status = state

    def reset_authenticators(self):
        self.auth = {
            'touch': TouchAuth(),
            'gait': DeepGaitAuthenticator(profile={'batch_size': 128, 'epochs': 25})
        }


    # =====================================
    # User management functions
    # =====================================
    def add_user(self, user_id):
        if user_id in self.user_profile:
            print("User already registered.")
            return
        self.user_profile.append(user_id)
        self.storage.add_user(user_id)

    def get_user_label(self, user_id):
        # generate user label dict
        user_label_dict = {}
        for i, user in enumerate(self.user_profile):
            user_label_dict[user] = i + 1
        if user_id not in user_label_dict:
            return 0
        else:
            return user_label_dict[user_id]

    def enroll(self, user_id, details):
        self.add_user(user_id)
        for item in details:
            # print(item['user'], user_id)
            assert item['user'] == user_id
            self.storage.add_data(user_id, item['type'], item['feature'])
        self.reset_authenticators()
        self.train_all()


    # =====================================
    # Utility functions for experiments
    # =====================================

    def save_decision(self, dem_name, out_path):
        with open(os.path.join(out_path, dem_name + ".txt"), 'w') as fp:
            for time, result in self.detection_events[dem_name]:
                fp.write(str(time) + "," + str(result) + "\n")

    def save_all_decisions(self, out_path):
        for dem_name in SUPPORTED_DECISION_MAKER.keys():
            self.save_decision(dem_name, out_path)

    def set_flag_store_history(self, b):
        self.flag_store_history = b

    def __init__(self, authenticators, auth_report=None, imposter_set=None, config=DEFAULT_CONFIGURATION):
        self.auth = authenticators
        self.auth_report = {} if auth_report is None else auth_report
        self.status = STATE_NORMAL # default normal state
        self.flag_store_history = True if config['model_update'] is None else config['model_update']
        self.current_user = USER_UNKNOWN
        self.previous_user = USER_UNKNOWN
        self.last_switch_time = -1
        self.cur_time = 0
        self.QUESTIONABLE_SWITCH_INTERVAL = 20000 # 20s user switch
        self.buffer = {
            'acc': [],
            'gyro': [],
            'touch': []
        }

        self.last_update = {
            'acc': -1,
            'gyro': -1,
            'touch': -1
        }

        self.detect_ctr = {
            'gait': 0
        }

        self.cur_swipe = -1

        self.decision_buffer = []
        self.another_buffer = []

        self.decision_scores = {
            'touch': [],
            'gait': [],
        }

        for dem in SUPPORTED_SCORER.keys():
            self.decision_scores[dem] = []


        ## dynamic system related variables
        self.biometric_cache = {
            'touch': [],
            'gait': []
        }


        if config['storage'] == 'random':
            self.storage = RandomStorage(
                auth_types= ['touch','gait'],
                imposter_set=imposter_set,
                max_space={
                    'touch': 50,
                    'gait': 500
                }
            )
            print("Use Random Storage")
        elif config['storage'] == 'distance':
            self.storage = MeanDistanceStorage(
                auth_types= ['touch','gait'],
                imposter_set=imposter_set,
                max_space={
                    'touch': 50,
                    'gait': 500
                }
            )
            print("Use Distance Storage")
        elif config['storage'] == 'unlimited':
            self.storage = UnBoundedStorage(
                auth_types= ['touch','gait'],
                imposter_set=imposter_set,
                max_space={
                    'touch': 50,
                    'gait': 500
                } # for consistency, it does not work
            )
            print("Use Unbounded Storage")
        else:
            self.storage = UnBoundedStorage(
                auth_types= ['touch','gait'],
                imposter_set=imposter_set,
                max_space={
                    'touch': 50,
                    'gait': 500
                } # for consistency, it does not work
            )
            print("Use Unbounded Storage")


        self.user_profile = []
        self.accept_signal = False if config['accept_user_change_signal'] is None else\
            config['accept_user_change_signal']
        if config['ext_sliding_window'] is not None:
            a, b = config['ext_sliding_window']
            self.decision_sw = SlidingWindow(a, b)
        else:
            self.decision_sw = SlidingWindow(7, 4)

        self.primary_dm = 'auc_dempster_shafer' \
          if config['decision_maker'] is None else config['decision_maker']

        print("Player Set up: ", self.primary_dm, self.flag_store_history, self.accept_signal)



In [ ]:

# scripter runner
# to handle exceptions during a simulation
# keep record of all progress
# calculate metrics of a simulation

## basic script workflow
## 1. obtain next event from timeline
## 2. feed it into the player
## 3. obtain detection result
## 4. update simulation status
## 5. capture exceptions
## 6. feedback

## whenever gets a valid result from player, first chech if it matches groundtruth

## some important definitions
## 1. false acceptance: false acceptance should be counted at session-level,

## when the user has changed (groundtruth) but the system
## is late to reject a stranger or identify the true user


## about metrics

EXCEPTION_FALSE_ACCEPTANCE = "FA"
EXCEPTION_FALSE_REJECTION = "FN"
EXCEPTION_MISCLASSIFICATION = "MC"


def generate_enrollment_data(provider, session_info):
    user = session_info[0]
    print("gen", user)
    d_sessions = session_info[1:]
    gait_sessions = [d_sessions[i + 1] for i in range(0, len(d_sessions), 2)
                     if d_sessions[i] == 'gait']
    touch_sessions = [d_sessions[i + 1] for i in range(0, len(d_sessions), 2)
                     if d_sessions[i] == 'touch']

    combined = []
    for sess in gait_sessions:
        feature = get_features(provider, 'gait', user, sess)
        combined.append({
            'user': user,
            'type': 'gait',
            'feature': feature
        })
    for sess in touch_sessions:
        feature = get_features(provider, 'touch', user, sess)
        combined.append({
            'user': user,
            'type': 'touch',
            'feature': feature
        })
    return combined

def script_runner(timeline: Timeline, player: TestPlayer):

    ## initialize the runner status
    cur_user = USER_UNKNOWN

    session_results = []
    tmp = None
    gt_start_time = 0
    decisions = []

    for event in timeline:
        result = REACTION_NONE
        if event[1] not in [EVENT_USER_CHANGE, EVENT_ADD_USER]:
            # EVENT_USER_CHANGE IS NOT FOR PLAYER
            result = player.dispatch(event)

        true_user = timeline.get_groundtruth_result(event[0])


        if event[1] in [EVENT_SESSION_END, EVENT_USER_CHANGE]:
            logger.info("[groundtruth] detect session end")
            # print("tmp", tmp)
            processed = process_session_result(tmp)
            if processed is not None:
                session_results.append(processed)
            tmp = None
        if event[1] in [EVENT_SESSION_START, EVENT_USER_CHANGE]:
            logger.info("[groundtruth] detect session start")
            tmp = {
                'num_detection': 0,
                'num_accept': 0,
                'num_reject': 0,
                'num_fp': 0,
                'num_fn': 0,
                'num_mc': 0, # misclassification
                'delay': -1,
                'exceptions': []
            }
            gt_start_time = event[0]
        if event[1] == EVENT_ADD_USER:
            ## explicit enrollment format
            ## name, session
            logger.info("[new user enrolment] user: " + str(event[2][0]))
            enroll_data = generate_enrollment_data(hmog, event[2])
            player.enroll(event[2][0], enroll_data)

        ## possible results:
        ## reaction types
        ## REACTION_NONE = 0
        ## REACTION_REJECT = -1
        ## REACTION_USER_CHANGE = 9
        ## REACTION_ACCEPT = 1

        if result == REACTION_ACCEPT:

            ## compare to ground truth
            tmp['num_detection'] += 1
            tmp['num_accept'] += 1
            if player.current_user != true_user:
                # current user
                if true_user not in player.user_profile:
                    # false acceptance
                    tmp['num_fp'] += 1
                    tmp['exceptions'].append((event[0], EXCEPTION_FALSE_ACCEPTANCE))
                    player.feedback((COMMENT_FALSE_ACCEPTANCE, USER_STRANGER))
                else:
                    # misclassification
                    tmp['num_mc'] += 1
                    tmp['exceptions'].append((event[0], EXCEPTION_MISCLASSIFICATION))
                    # player.feedback((COMMENT_FALSE_MISCLASSIFICATION, true_user))
            else:
                # true positive
                # check if it is the first detection
                if tmp['delay'] == -1:
                    tmp['delay'] = event[0] - gt_start_time

        elif result == REACTION_REJECT:
            tmp['num_detection'] += 1
            tmp['num_reject'] += 1
            print("simulator says: falsely rejected me!")
            if true_user in player.user_profile:
                # false rejection
                tmp['num_fn'] += 1
                tmp['exceptions'].append((event[0], EXCEPTION_FALSE_REJECTION))
                player.feedback((COMMENT_FALSE_REJECTION, true_user))
            else:
                # true rejection (check first detection)
                if tmp['delay'] == -1:
                    tmp['delay'] = event[0] - gt_start_time

        elif result == REACTION_USER_CHANGE:
            print("simulator says: I hear about user change")
            tmp['num_detection'] += 1
            # check if it is really a user change
            tmp['num_accept'] += 1
            if true_user == player.current_user:
                # yes!
                print("simulator says: ...& its true!")
                player.feedback((COMMENT_CONFIRM_USER_CHANGE, None))

            else:
                if true_user not in player.user_profile:
                    # false acceptance
                    tmp['num_fp'] += 1
                    tmp['exceptions'].append((event[0], EXCEPTION_FALSE_ACCEPTANCE))
                    # player.feedback((COMMENT_FALSE_ACCEPTANCE, USER_STRANGER))
                else:
                    print("simulator says: ... but its wrong!", true_user)
                    tmp['num_mc'] += 1
                    tmp['exceptions'].append((event[0], EXCEPTION_MISCLASSIFICATION))
                    player.feedback((COMMENT_DENY_USER_CHANGE, true_user))
                    # player.feedback((COMMENT_FALSE_MISCLASSIFICATION, true_user))

    return session_results


def process_session_result(tmp):
    if tmp is None:
        return None
    return tmp

In [ ]:
def script_runner_v2(timeline: Timeline, player: TestPlayer):

    ## initialize the runner status
    cur_user = USER_UNKNOWN
    uninformed_interval = 512000

    session_results = []
    tmp = None
    gt_start_time = 0
    decisions = []
    user_switch_start_time = -1

    for event in timeline:
        result = REACTION_NONE
        if event[1] not in [EVENT_ADD_USER]:
            result = player.dispatch(event)

        true_user = timeline.get_groundtruth_result(event[0])


        if event[1] in [EVENT_SESSION_END, EVENT_USER_CHANGE]:
            logger.info("[groundtruth] detect session end")
            if event[1] == EVENT_SESSION_END:
                user_switch_start_time = -1
            # print("tmp", tmp)
            processed = process_session_result(tmp)
            if processed is not None:
                session_results.append(processed)
            tmp = None
        if event[1] in [EVENT_SESSION_START, EVENT_USER_CHANGE]:
            if event[1] == EVENT_USER_CHANGE:
                user_switch_start_time = event[0]

            logger.info("[groundtruth] detect session start")
            tmp = {
                'num_detection': 0,
                'num_accept': 0,
                'num_reject': 0,
                'num_fp': 0,
                'num_fn': 0,
                'num_mc': 0, # misclassification
                'delay': -1,
                'exceptions': []
            }
            gt_start_time = event[0]
        if event[1] == EVENT_ADD_USER:
            ## explicit enrollment format
            ## name, session
            logger.info("[new user enrolment] user: " + str(event[2][0]))
            enroll_data = generate_enrollment_data(hmog, event[2])
            player.enroll(event[2][0], enroll_data)

        ## possible results:
        ## reaction types
        ## REACTION_NONE = 0
        ## REACTION_REJECT = -1
        ## REACTION_USER_CHANGE = 9
        ## REACTION_ACCEPT = 1

        if result == REACTION_ACCEPT:

            ## compare to ground truth
            tmp['num_detection'] += 1
            tmp['num_accept'] += 1
            if player.current_user != true_user:
                # current user
                if true_user not in player.user_profile:
                    # false acceptance
                    tmp['num_fp'] += 1
                    tmp['exceptions'].append((event[0], EXCEPTION_FALSE_ACCEPTANCE))
                    player.feedback((COMMENT_FALSE_ACCEPTANCE, USER_STRANGER))
                else:
                    # misclassification
                    if user_switch_start_time < 0 or event[0] - user_switch_start_time > uninformed_interval:
                        tmp['num_mc'] += 1
                        tmp['exceptions'].append((event[0], EXCEPTION_MISCLASSIFICATION))
                    # player.feedback((COMMENT_FALSE_MISCLASSIFICATION, true_user))
            else:
                # true positive
                # check if it is the first detection
                if tmp['delay'] == -1:
                    tmp['delay'] = event[0] - gt_start_time

        elif result == REACTION_REJECT:
            tmp['num_detection'] += 1
            tmp['num_reject'] += 1
            print("simulator says: falsely rejected me!")
            if true_user in player.user_profile:
                # false rejection
                tmp['num_fn'] += 1
                tmp['exceptions'].append((event[0], EXCEPTION_FALSE_REJECTION))
                player.feedback((COMMENT_FALSE_REJECTION, true_user))
            else:
                # true rejection (check first detection)
                if tmp['delay'] == -1:
                    tmp['delay'] = event[0] - gt_start_time

        elif result == REACTION_USER_CHANGE:
            print("simulator says: I hear about user change")
            tmp['num_detection'] += 1
            # check if it is really a user change
            tmp['num_accept'] += 1
            if true_user == player.current_user:
                # yes!
                print("simulator says: ...& its true!")
                player.feedback((COMMENT_CONFIRM_USER_CHANGE, None))

            else:
                if true_user not in player.user_profile:
                    # false acceptance
                    tmp['num_fp'] += 1
                    tmp['exceptions'].append((event[0], EXCEPTION_FALSE_ACCEPTANCE))
                    # player.feedback((COMMENT_FALSE_ACCEPTANCE, USER_STRANGER))
                else:
                    print("simulator says: ... but its wrong!", true_user)
                    tmp['num_mc'] += 1
                    tmp['exceptions'].append((event[0], EXCEPTION_MISCLASSIFICATION))
                    player.feedback((COMMENT_DENY_USER_CHANGE, true_user))
                    # player.feedback((COMMENT_FALSE_MISCLASSIFICATION, true_user))

    return session_results

In [ ]:
def gen_groundtruth_for_decision_scores(tml: Timeline, player: TestPlayer):
    # generate groundtruth table for decision scores
    # return {<decison_maker>: {'pred': <scores>, 'true': <groundtruth labels>}}
    result_dict = {}
    for dec_name, dec_list in player.decision_scores.items():
        # process for a certain dec
        grt_list = []
        pred_list = []
        for time, result in dec_list:
            user = tml.get_groundtruth_result(time)
            grt = 0 if user not in player.user_profile else player.get_user_label(user)
            grt_list.append(grt)
            pred_list.append(result)
        result_dict[dec_name] = {
            'pred': pred_list,
            'true': grt_list
        }
    return result_dict


def one_vs_rest_binarize(decision_pair, pos_loc):
    score = np.array(decision_pair['pred'])[:, pos_loc]
    grt = (np.array(decision_pair['true']) == pos_loc).astype(int)
    full_score = np.array([[1 - s, s] for s in score])
    return {
        'score': score,
        'true': grt,
        'pred': full_score
    }

def one_vs_one_binarize(decision_pair, pos_loc, neg_loc):
    score = np.array(decision_pair['pred'])
    truth = np.array(decision_pair['true'])
    sel_score = score[(truth == pos_loc) | (truth == neg_loc)]
    sel_truth = truth[(truth == pos_loc) | (truth == neg_loc)]
    grt = (sel_truth == pos_loc).astype(int)
    single_score = np.array([s[pos_loc] for s in sel_score])
    full_score = np.array([[s[neg_loc], s[pos_loc]] for s in sel_score])
    return {
        'score': single_score,
        'true': grt,
        'pred': full_score
    }


def gen_per_segment_decision(tml: Timeline, player: TestPlayer):
    result_dict = {}
    for dec_name, dec_list in player.decision_scores.items():
        grt_list = []
        pred_list = []
        time_list = []
        segment_no = 0
        st, ed, user, _ = tml.user_groundtruth[segment_no]
        seg_grt = []
        seg_pred = []
        seg_time = []
        for time, result in dec_list:
            # user = tml.get_groundtruth_result(time)
            assert time >= st
            if time > ed:
                # move to the next segment
                grt_list.append(seg_grt)
                pred_list.append(seg_pred)
                time_list.append(seg_time)
                seg_grt = []
                seg_pred = []
                seg_time = []
                segment_no += 1
                assert segment_no <= len(tml.user_groundtruth)
                st, ed, user, _ = tml.user_groundtruth[segment_no]
            grt = 0 if user not in player.user_profile else player.get_user_label(user)
            seg_grt.append(grt)
            seg_pred.append(result)
            seg_time.append(time - st)
        grt_list.append(seg_grt)
        pred_list.append(seg_pred)
        time_list.append(seg_time)
        result_dict[dec_name] = {
            'pred': pred_list,
            'true': grt_list,
            'time': time_list
        }
    return result_dict



def get_metrics_per_segment(decision_pair, sliding_window=None):
    positives = 0
    negatives = 0
    false_rejections = 0
    true_rejections = 0
    false_identifications = 0
    latency_rejection = []
    result_dict = {
        'pos': 0,
        'neg': 0,
        'fr': 0,
        'tr': 0,
        'fi': 0,
        'latency': []
    }

    for i, j, t in zip(decision_pair['pred'], decision_pair['true'],
                      decision_pair['time']):
        # print(i)
        # print(j)
        score = np.array(i)
        truth = np.array(j)
        result = np.argmax(score, axis=1)
        if sum(truth) > 0:
            result_dict['pos'] += 1
            # positive case, focus on false rejection and false identification
            if sliding_window is not None:
                m, n = sliding_window
                sw = SlidingWindow(m, n)
                # stop at first rejection or misclassification
                flag = False
                for res in result:
                    decision = sw.add(res)
                    if decision == 0:
                        result_dict['fr'] += 1
                        flag = True
                        break
                    elif decision != truth[0] and decision > 0:
                        print(decision, truth[0])
                        result_dict['fi'] += 1
                        flag = True
                        break
            else:
                for res in result:
                    if res == 0:
                        result_dict['fr'] += 1
                        flag = True
                        break
                    elif res != truth[0]:
                        print(res, truth[0])
                        result_dict['fi'] += 1
                        flag = True
                        break
        else:
            result_dict['neg'] += 1
            if sliding_window is not None:
                m, n = sliding_window
                sw = SlidingWindow(m, n)
                # stop at first rejection or misclassification
                flag = False
                for res, time in zip(result, t):
                    decision = sw.add(res)
                    if decision == 0:
                        result_dict['tr'] += 1
                        result_dict['latency'].append(time)
                        flag = True
                        break
            else:
                for res, time in zip(result, t):
                    if res == 0:
                        result_dict['tr'] += 1
                        result_dict['latency'].append(time)
                        flag = True
                        break
    return result_dict


def gen_all_metrics_for_scheme(dcp, num_valid_users=2):
    final_result = dict()
    for user in range(num_valid_users):
        per_user_result = dict()
        result = one_vs_rest_binarize(dcp, user + 1)

        per_user_result['true'] = result['true']
        per_user_result['score'] = result['score']
        per_user_result['pred'] = result['pred']

        # obtain roc curve
        fpr, tpr, thres = roc_curve(result['true'], result['score'])

        per_user_result['fpr'] = fpr
        per_user_result['tpr'] = tpr
        per_user_result['thres'] = thres

        eer_score, _ = eer(result['true'], result['score'])
        per_user_result['eer'] = eer_score
        per_user_result['auc'] = roc_auc_score(result['true'], result['score'])
        final_result[user + 1] = per_user_result
    return final_result

def gen_all_metrics(score_dict, num_valid_users=2):
    return {
        name: gen_all_metrics_for_scheme(dcp, num_valid_users)
        for name, dcp in score_dict.items()
    }


# metrics_all = gen_all_metrics(score_dict, 2)


# fcs = FCS('simple_average')
# fcs.get_metric(result['true'], result['pred'],
#                (result['score'] > 0.3).astype(int))



import seaborn as sns

class FCS():

    def __init__(self, classifier_name):
        self.classifier_name = classifier_name
        self.true_labels = None
        self.pred_probs = None
        self.pred_labels = None
        self.plot = None
        self.axes = None

        return

    def get_metric(self, true_labels, predicted_probs, pred_labels, bins="auto",
                   ax=None, alpha=0.75):
        self.true_labels = true_labels
        self.pred_probs = predicted_probs
        self.pred_labels = pred_labels
        self.axes = ax
        df = pd.DataFrame(self.pred_probs)
        df.columns = ['prob_0', 'prob_1']
        df['pred_labels'] = pred_labels
        df['true_labels'] = true_labels
        df.loc[df['true_labels'] == 1, 'user_type'] = "positive_user"
        df.loc[df['true_labels'] == 0, 'user_type'] = "negative_user"

        bins = bins
        if self.axes is None:
            # sns.set_theme("whitegrid")
            self.plot = plt.figure(figsize=[12, 12])
            self.axes = self.plot.add_subplot(1, 1, 1)
            # sns.histplot(x="prob_1", data=df, hue="user_type", bins=bins, ax=self.axes, legend=True)

            self.axes.hist(df[df['true_labels'] == 1].prob_1.values, bins=bins,
                           label="Positive User",
                           edgecolor='black', linewidth=1.2, alpha=alpha)
            self.axes.hist(df[df['true_labels'] == 0].prob_1.values, bins=bins,
                           label="Negative User",
                           edgecolor='black', linewidth=1.2, alpha=alpha)

            self.axes.set_title('Frequency Count Score for ' + self.classifier_name)
            self.axes.set_xlabel('scores')
            self.axes.set_ylabel('Frequency Count')

            return self.plot
        else:
            pass
            # sns.histplot(x="prob_1", data=df, hue="user_type", bins=bins, ax=self.axes, legend=True)

            self.axes.hist(df[df['true_labels'] == 1].prob_1.values, bins=bins, label="Positive User",
                           edgecolor='black', linewidth=1.2, alpha=alpha)
            self.axes.hist(df[df['true_labels'] == 0].prob_1.values, bins=bins, label="Negative User",
                           edgecolor='black', linewidth=1.2, alpha=alpha)

            self.axes.set_title('Frequency Count Score for ' + self.classifier_name)
            self.axes.set_xlabel('scores')
            self.axes.set_ylabel('Frequency Count')

        return


In [ ]:
### save experiment data

def save_experiment_data(player, tml, path):
    if os.path.exists(path) and os.path.isdir(path):
        print("Path exists, will overwrite the path")
    else:
        os.makedirs(path)

    # save timeline data
    tml.save("timeline", path)

    # save decisions
    # player.decision_scores, player.user_profiles
    # put everything into a big dictionary
    json_dict = {
        'user': player.user_profile,
        'decisions': []
    }
    for key, score in player.decision_scores.items():
        json_dict['decisions'].append(
            {
                'strategy': key,
                'result': [
                          {'time': time, 'score': s.tolist()}
                          for time, s in score
                ]
            }
        )

    # print(json_dict)

    with open(os.path.join(path, "result.json"), "w") as outfile:
        json.dump(json_dict, outfile)

    print("file saved.")

In [ ]:
def segment_by_session_end(tml):
    st = 0
    segments = []
    for time, event, content in tml.events:
        if event == EVENT_SESSION_END:
            segments.append((st, time))
            st = time
    return segments

# print(segment_by_session_end(tml))

def gen_decisions_by_segment(tml, player, segments):
    result_dict = {}
    for dec_name, dec_list in player.decision_scores.items():
        grt_list = []
        pred_list = []
        time_list = []
        segment_no = 0
        st, ed = segments[segment_no]
        seg_grt = []
        seg_pred = []
        seg_time = []
        for time, result in dec_list:
            user = tml.get_groundtruth_result(time)
            assert time >= st
            if time > ed:
                grt_list.append(seg_grt)
                pred_list.append(seg_pred)
                time_list.append(seg_time)
                seg_grt = []
                seg_pred = []
                seg_time = []
                segment_no += 1
                st, ed = segments[segment_no]
            grt = 0 if user not in player.user_profile else player.get_user_label(user)
            seg_grt.append(grt)
            seg_pred.append(result)
            seg_time.append(time - st)
        grt_list.append(seg_grt)
        pred_list.append(seg_pred)
        time_list.append(seg_time)
        result_dict[dec_name] = {
            'pred': pred_list,
            'true': grt_list,
            'time': time_list
        }
    return result_dict


def get_metrics_for_experiment2(decision_pair, sliding_window=None):
    ## types of errors:
    ## early failures: early rejection and early misclassification
    ## identification failures: 1. false rejection 2. misclassification
    ## there is no negative case

    result_dict = {
        'case': 0,
        'er': 0,
        'em': 0,
        'fr': 0,
        'mi': 0,
        'latency': []
    }

    # owner = decision_pair['true'][0]
    # sharee = decision_pair['true'][-1]

    # moment = 0
    # for i, j in zip(decision_pair['true'], decision_pair['time']):
    #     if i == sharee:
    #         moment = j
    #         break


    for i, j, t in zip(decision_pair['pred'], decision_pair['true'],
                      decision_pair['time']):
        # print(i)
        # print(j)
        score = np.array(i)
        truth = np.array(j)
        # print(score, truth)
        result = np.argmax(score, axis=1)

        # obtain moment:
        owner = truth[0]
        sharee = truth[-1]
        moment = -1
        for i, j in zip(t, truth):
            if j == sharee:
                moment = i
                break
        assert moment != -1

        result_dict['case'] += 1

        if sliding_window is not None:
            m, n = sliding_window
            sw = SlidingWindow(m, n)
            early_rejection = False
            early_misclassification = False
            false_rejection = False
            misclassification = False
            flag = False
            for res, tru, tim in zip(result, truth, t):
                decision = sw.add(res)
                if tru == owner:
                    # pre sharing moment
                    if decision == 0: # false rejection
                        early_rejection = True
                        # print(tim, decision)
                    elif decision != owner and decision > 0:
                        early_misclassification = True
                elif tru == sharee:
                    if decision == 0:
                        false_rejection = True
                    # if decision == owner and :
                    #     misclassification = True
                    if decision != sharee and tim - moment > 60000 and decision > 0:
                        misclassification = True
                    if not flag and decision == sharee:
                        result_dict['latency'].append(tim - moment)
                        flag = True

            if early_rejection:
                result_dict['er'] += 1
            elif early_misclassification:
                result_dict['em'] += 1
            elif false_rejection:
                result_dict['fr'] += 1
            elif misclassification:
                result_dict['mi'] += 1


    return result_dict


def gen_per_segment_decision(tml: Timeline, player: TestPlayer):
    result_dict = {}
    for dec_name, dec_list in player.decision_scores.items():
        grt_list = []
        pred_list = []
        time_list = []
        segment_no = 0
        st, ed, user, _ = tml.user_groundtruth[segment_no]
        seg_grt = []
        seg_pred = []
        seg_time = []
        for time, result in dec_list:
            # user = tml.get_groundtruth_result(time)
            assert time >= st
            if time > ed:
                # move to the next segment
                grt_list.append(seg_grt)
                pred_list.append(seg_pred)
                time_list.append(seg_time)
                seg_grt = []
                seg_pred = []
                seg_time = []
                segment_no += 1
                assert segment_no <= len(tml.user_groundtruth)
                st, ed, user, _ = tml.user_groundtruth[segment_no]
            grt = 0 if user not in player.user_profile else player.get_user_label(user)
            seg_grt.append(grt)
            seg_pred.append(result)
            seg_time.append(time - st)
        grt_list.append(seg_grt)
        pred_list.append(seg_pred)
        time_list.append(seg_time)
        result_dict[dec_name] = {
            'pred': pred_list,
            'true': grt_list,
            'time': time_list
        }
    return result_dict



def get_metrics_per_segment(decision_pair, sliding_window=None):
    positives = 0
    negatives = 0
    false_rejections = 0
    true_rejections = 0
    false_identifications = 0
    latency_rejection = []
    result_dict = {
        'pos': 0,
        'neg': 0,
        'fr': 0,
        'tr': 0,
        'fi': 0,
        'latency': []
    }

    for i, j, t in zip(decision_pair['pred'], decision_pair['true'],
                      decision_pair['time']):
        # print(i)
        # print(j)
        score = np.array(i)
        truth = np.array(j)
        # print(score, truth)
        if len(score) == 0:
            continue

        result = np.argmax(score, axis=1)
        if sum(truth) > 0:
            result_dict['pos'] += 1
            # positive case, focus on false rejection and false identification
            if sliding_window is not None:
                m, n = sliding_window
                sw = SlidingWindow(m, n)
                # stop at first rejection or misclassification
                flag = False
                for res in result:
                    decision = sw.add(res)
                    if decision == 0:
                        result_dict['fr'] += 1
                        flag = True
                        break
                    elif decision != truth[0] and decision > 0:
                        print(decision, truth[0])
                        result_dict['fi'] += 1
                        flag = True
                        break
            else:
                for res in result:
                    if res == 0:
                        result_dict['fr'] += 1
                        flag = True
                        break
                    elif res != truth[0]:
                        print(res, truth[0])
                        result_dict['fi'] += 1
                        flag = True
                        break
        else:
            result_dict['neg'] += 1
            if sliding_window is not None:
                m, n = sliding_window
                sw = SlidingWindow(m, n)
                # stop at first rejection or misclassification
                flag = False
                for res, time in zip(result, t):
                    decision = sw.add(res)
                    if decision == 0:
                        result_dict['tr'] += 1
                        result_dict['latency'].append(time)
                        flag = True
                        break
            else:
                for res, time in zip(result, t):
                    if res == 0:
                        result_dict['tr'] += 1
                        result_dict['latency'].append(time)
                        flag = True
                        break
    return result_dict


In [ ]:
## initialize hmog dataset provider
hmog = HMOGProvider("drive/MyDrive/dataset/HMOG")

In [ ]:
## Negative Trainer (ImposterProvider)
trainer_seed = 100
num_trainers = 10

## Step 1: obtain trainers
trainers = hmog.gen_user_set(num_trainers, random_state=trainer_seed)

## Step 2: load data
hmog.load(trainers)

## Step 3: obtain session data for each authenticator

### Step 3.1: gait
gait_set = []
for user in trainers:
    ## gait sessions
    sel_sessions = hmog.get_sessions_for_unloaded_user(user,
                                [hmog.ACTIVITY_READING_WALKING,
                                 hmog.ACTIVITY_WRITING_WALKING])
    result = None
    for session in sel_sessions:
        tmp_data = simple_merge_acc_gyro(hmog.data[user][session]['acc'],
                                         hmog.data[user][session]['gyro'])
        features = DeepGaitAuthenticator().extract_features(tmp_data)
        if features is None:
            print(user, session, "contains no gait")
            continue
        result = features if result is None else np.concatenate(
            [result, features])
    gait_set.append(result)

### Step 3.2: touch
touch_set = []
for user in trainers:
    ## gait sessions
    sel_sessions = hmog.get_sessions_for_unloaded_user(user,
                                [hmog.ACTIVITY_READING_WALKING,
                                 hmog.ACTIVITY_READING_SITTING])
    result = None
    for session in sel_sessions:
        # print(user, session)
        features = touch_df_to_features(hmog.data[user][session]['touch'])
        # if user == '219303' and int(session) == 17:
        #     print(len(features))
        result = features if result is None else np.concatenate(
            [result, features])
    touch_set.append(result)

## Step4: generate imposter set
gait_imposters = ImposterProvider(gait_set)
gait_train, gait_test = gait_imposters.generate_train_test_sets(2)
touch_imposters = ImposterProvider(touch_set)
touch_train, touch_test = touch_imposters.generate_train_test_sets(2)
# print(gait_train.shape, gait_test.shape, touch_train.shape, touch_test.shape)
imposter_set = {
    'touch': {'train': touch_train, 'test': touch_test},
    'gait': {'train': gait_train, 'test': gait_test}
}

In [ ]:
dge = tf.keras.models.load_model("drive/MyDrive/deep_gait_extraction_21")

In [ ]:
def run_topic_robustness_type_cmp(game_seed=0):
     # ================================================
    # 1. SETUP A GAME AND VIEW BASIC INFORMATION
    # ================================================


    num_testers = 0
    num_valid_users = 3

    testers = hmog.gen_user_set(num_testers, exclude=trainers,
                                random_state=game_seed)
    valid_users = hmog.gen_user_set(num_valid_users,
                                    exclude=trainers + testers,
                                    random_state=game_seed)

    all_users = trainers + testers + valid_users

    train_test_dict = hmog.gen_multiuser_train_test_sessions(all_users, 1,
                                                             random_state=game_seed)
    game_name = "robust" + "_".join([str(trainer_seed), str(game_seed), str(num_trainers),
                                     str(num_testers), str(num_valid_users)])


    summary = "========== SETUP ===========\n"
    summary += "Game name:" + game_name + "\n"
    summary += "Negative trainer seed:" + str(trainer_seed) + "\n"
    summary += "Game seed:" + str(game_seed) + "\n"
    summary += "trainers:" + ", ".join(trainers) + "\n"
    summary += "users:" + ", ".join([u + " (" + str(i) + ")"
                                     for i, u in enumerate(valid_users)]) + "\n"
    summary += "testers:" + ", ".join(testers) + "\n"

    # ================================================
    # 1.5 SHOW GAME DETAILS (SESSION DISTRIBUTION)
    # ================================================

    if GAME_DETAILS_ENABELD:
        summary += "\n========== DETAILS ===========\n"
        for user, train_test in train_test_dict.items():
            summary += user + "\n"
            summary += "  Training sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['train'][no]]) + "\n"
            summary += "  Testing sessions: \n"
            for no, activity in ACTIVITIES:
                summary += "    " + activity + ": " + ", ".join(
                    [str(session) for session in
                     train_test['test'][no]]) + "\n"

    print(summary)

    # ===============================================
    # 2. LOAD DATA AND PRETRAINED MODELS
    # ===============================================

    print("Load data to memory. It may take up to 5 minutes...")
    hmog.load(all_users)

    # player = TestPlayer(authenticators={
    #     'touch': TouchAuth(),
    #     'gait': DeepGaitAuthenticator(profile={'batch_size': 128, 'epochs': 25})
    # }, imposter_set=imposter_set,
    # config={
    #     'decision_maker': 'auc_dempster_shafer',
    #     'ext_sliding_window': (7, 4),
    #     'model_update': True,
    #     'accept_user_change_signal': False,
    # }
    # )


    # player2 = TestPlayer(authenticators={
    #     'touch': TouchAuth(),
    #     'gait': DeepGaitAuthenticator(profile={'batch_size': 128, 'epochs': 25})
    # }, imposter_set=imposter_set,
    # config={
    #     'decision_maker': 'auc_dempster_shafer',
    #     'ext_sliding_window': (7, 4),
    #     'model_update': False,
    #     'accept_user_change_signal': False,
    # })

    player = TestPlayer(authenticators={
        'touch': TouchAuth(),
        'gait': DeepGaitAuthenticator(profile={'batch_size': 256, 'epochs': 25})
    }, imposter_set=imposter_set,
    config={
        'decision_maker': 'auc_dempster_shafer',
        'ext_sliding_window': (7, 4),
        'model_update': True,
        'accept_user_change_signal': True,
        'storage': 'distance',
    })

    player2 = TestPlayer(authenticators={
        'touch': TouchAuth(),
        'gait': DeepGaitAuthenticator(profile={'batch_size': 256, 'epochs': 25})
    }, imposter_set=imposter_set,
    config={
        'decision_maker': 'auc_dempster_shafer',
        'ext_sliding_window': (7, 4),
        'model_update': True,
        'accept_user_change_signal': True,
        'storage': 'random',
    })

    player3 = TestPlayer(authenticators={
        'touch': TouchAuth(),
        'gait': DeepGaitAuthenticator(profile={'batch_size': 256, 'epochs': 25})
    }, imposter_set=imposter_set,
    config={
        'decision_maker': 'auc_dempster_shafer',
        'ext_sliding_window': (7, 4),
        'model_update': True,
        'accept_user_change_signal': True,
        'storage': 'unlimited',
    })

    # player2.set_flag_store_history(False) # player2 never learn

    # ===============================================
    # 3. SCRIPT INIT
    # ===============================================
    enrollment_script_type_1 = []

    # for user in valid_users:
    #     gait_sessions = hmog.get_sessions_for_unloaded_user(
    #         user,[hmog.ACTIVITY_READING_WALKING])[:n_train_session] \
    #         + hmog.get_sessions_for_unloaded_user(
    #         user,[hmog.ACTIVITY_WRITING_WALKING])[:n_train_session]

    #     touch_sessions = hmog.get_sessions_for_unloaded_user(
    #         user, [hmog.ACTIVITY_READING_SITTING])[:n_train_session] \
    #         + hmog.get_sessions_for_unloaded_user(
    #         user, [hmog.ACTIVITY_READING_WALKING])[:n_train_session]

    #     enrollment_script_type_1.append((user, 'gait', gait_sessions))
    #     enrollment_script_type_1.append((user, 'touch', touch_sessions))

    ## FOR OWNER
    user = valid_users[0]
    gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:2]

    touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:2] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:1]

    enrollment_script_type_1.append((user, 'gait', gait_sessions))
    enrollment_script_type_1.append((user, 'touch', touch_sessions))

    ## FOR VALID USER 1
    user = valid_users[1]
    gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:1]

    touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:1]

    enrollment_script_type_1.append((user, 'gait', gait_sessions))
    enrollment_script_type_1.append((user, 'touch', touch_sessions))

    tf.random.set_seed(game_seed)
    for user, auth_type, sessions in enrollment_script_type_1:
        # print("try to add", user)
        player.add_user(user)
        for sess in sessions:
            feature = get_features(hmog, auth_type, user, sess)
            # print(player.storage.data.keys())
            player.storage.add_data(user, auth_type, feature)
    # print(player.user_profile)
    player.train_all()
    print("training done")

    tf.random.set_seed(game_seed)
    for user, auth_type, sessions in enrollment_script_type_1:
        # print("try to add", user)
        player2.add_user(user)
        for sess in sessions:
            feature = get_features(hmog, auth_type, user, sess)
            # print(player.storage.data.keys())
            player2.storage.add_data(user, auth_type, feature)
    # print(player.user_profile)
    player2.train_all()
    print("training done")


    tf.random.set_seed(game_seed)
    for user, auth_type, sessions in enrollment_script_type_1:
        # print("try to add", user)
        player3.add_user(user)
        for sess in sessions:
            feature = get_features(hmog, auth_type, user, sess)
            # print(player.storage.data.keys())
            player3.storage.add_data(user, auth_type, feature)
    # print(player.user_profile)
    player3.train_all()
    print("training done")


    tml = Timeline()
    # ===============================================
    # 4. STORY
    # ===============================================


    np.random.seed(0)
    lengths = np.random.randint(120000,300000, 20)


    # Part 1: owner - user 1

    user = valid_users[0]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[1]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[0])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_USER_CHANGE, [])

    user = valid_users[1]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[1]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[1])
    # tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])
    tml.add_special_event(EVENT_TRAIN_MODELS, [])

    # Part 2: new user
    user = valid_users[2]
    gait_sessions = hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_READING_WALKING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user,[hmog.ACTIVITY_WRITING_WALKING])[:1]

    touch_sessions = hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_SITTING])[:1] \
            + hmog.get_sessions_for_unloaded_user(
            user, [hmog.ACTIVITY_READING_WALKING])[:1]

    user_info = [user]
    for session in gait_sessions:
        user_info += ['gait', session]
    for session in touch_sessions :
        user_info += ['touch', session]
    tml.add_special_event(EVENT_ADD_USER, user_info)

    user = valid_users[0]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[2]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[2])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    # tml.add_special_event(EVENT_SESSION_END, [])
    tml.add_special_event(EVENT_USER_CHANGE, [])

    user = valid_users[1]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[2]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[3])
    # tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    # tml.add_special_event(EVENT_SESSION_END, [])
    tml.add_special_event(EVENT_USER_CHANGE, [])

    user = valid_users[2]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[1]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[4])
    # tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])
    tml.add_special_event(EVENT_TRAIN_MODELS, [])

    ## Part 3 New data
    user = valid_users[0]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user, [hmog.ACTIVITY_READING_WALKING])[3]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[5])
    tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    # tml.add_special_event(EVENT_SESSION_END, [])
    tml.add_special_event(EVENT_USER_CHANGE, [])

    user = valid_users[1]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user, [hmog.ACTIVITY_READING_WALKING])[3]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[6])
    # tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    # tml.add_special_event(EVENT_SESSION_END, [])
    tml.add_special_event(EVENT_USER_CHANGE, [])

    user = valid_users[2]
    sel_session = hmog.get_sessions_for_unloaded_user(
                user,[hmog.ACTIVITY_READING_WALKING])[2]
    events = get_events_for_user_session_with_limit(user, sel_session, tml.end_time, lengths[7])
    # tml.add_special_event(EVENT_SESSION_START, [])
    tml.add_user_session(user, hmog.ACTIVITY_READING_WALKING, events)
    tml.add_special_event(EVENT_SESSION_END, [])



    # ===============================================
    # 5. One line running script
    # ===============================================
    np.random.seed(game_seed)
    tf.random.set_seed(game_seed)
    print("======= Player 1 Start ========")
    results = script_runner_v2(tml, player)
    np.random.seed(game_seed)
    tf.random.set_seed(game_seed)
    print("======= Player 2 Start ========")
    results2 = script_runner_v2(tml, player2)
    np.random.seed(game_seed)
    tf.random.set_seed(game_seed)
    print("======= Player 3 Start ========")
    results3 = script_runner_v2(tml, player3)

    # ===============================================
    # 6. Metrics and Report Saving
    # ===============================================

    total_result = {
        'num_detection': 0,
        'num_accept': 0,
        'num_reject': 0,
        'num_fn': 0,
        'num_fp': 0,
        'num_mc': 0,
        'delay': []
    }

    print("===> simulator 1 <===")
    for i, result in enumerate(results):
        to_show = "==== Session #" + str(i) + " ====\n"
        to_show += "# of detection: " + str(result['num_detection']) + ", "
        to_show += "acceptance: " + str(result['num_accept']) + ", "
        to_show += "rejection: " + str(result['num_reject']) + "\n"
        to_show += "# of FA: " + str(result['num_fp']) + "\n"
        to_show += "# of FR: " + str(result['num_fn']) + "\n"
        to_show += "# of MI: " + str(result['num_mc']) + "\n"
        if result['num_detection'] > 0:
            to_show += "FAR: " + str(result['num_fp'] / result['num_detection']) + "\n"
            to_show += "MIR: " + str(result['num_mc'] / result['num_detection']) + "\n"
            to_show += "FRR: " + str(result['num_fn'] / result['num_detection']) + "\n"

        to_show += "latency: " + str(result['delay']) + " ms"
        print(to_show)

        total_result['num_detection'] += result['num_detection']
        total_result['num_accept'] += result['num_accept']
        total_result['num_reject'] += result['num_reject']
        total_result['num_fp'] += result['num_fp']
        total_result['num_fn'] += result['num_fn']
        total_result['num_mc'] += result['num_mc']
        total_result['delay'].append(result['delay'])

    to_show = "simulation result: "
    if total_result['num_detection'] > 0:
        to_show += "FAR: " + str(total_result['num_fp'] / total_result['num_detection']) + "\n"
        to_show += "MIR: " + str(total_result['num_mc'] / total_result['num_detection']) + "\n"
        to_show += "FNR: " + str(total_result['num_fn'] / total_result['num_detection']) + "\n"
    to_show += "latency: " + str(np.average(total_result['delay'])) + " ms"
    print(to_show)

    # print("===> simulator 2 <===")

    # total_result = {
    #     'num_detection': 0,
    #     'num_accept': 0,
    #     'num_reject': 0,
    #     'num_fn': 0,
    #     'num_fp': 0,
    #     'num_mc': 0,
    #     'delay': []
    # }

    # for i, result in enumerate(results2):
    #     to_show = "==== Session #" + str(i) + " ====\n"
    #     to_show += "# of detection: " + str(result['num_detection']) + ", "
    #     to_show += "acceptance: " + str(result['num_accept']) + ", "
    #     to_show += "rejection: " + str(result['num_reject']) + "\n"
    #     to_show += "# of FA: " + str(result['num_fp']) + "\n"
    #     to_show += "# of FR: " + str(result['num_fn']) + "\n"
    #     to_show += "# of MI: " + str(result['num_mc']) + "\n"
    #     if result['num_detection'] > 0:
    #         to_show += "FAR: " + str(result['num_fp'] / result['num_detection']) + "\n"
    #         to_show += "MIR: " + str(result['num_mc'] / result['num_detection']) + "\n"
    #         to_show += "FRR: " + str(result['num_fn'] / result['num_detection']) + "\n"

    #     to_show += "latency: " + str(result['delay']) + " ms"
    #     print(to_show)

    #     total_result['num_detection'] += result['num_detection']
    #     total_result['num_accept'] += result['num_accept']
    #     total_result['num_reject'] += result['num_reject']
    #     total_result['num_fp'] += result['num_fp']
    #     total_result['num_fn'] += result['num_fn']
    #     total_result['num_mc'] += result['num_mc']
    #     total_result['delay'].append(result['delay'])

    # to_show = "simulation result: "
    # if total_result['num_detection'] > 0:
    #     to_show += "FAR: " + str(total_result['num_fp'] / total_result['num_detection']) + "\n"
    #     to_show += "MIR: " + str(total_result['num_mc'] / total_result['num_detection']) + "\n"
    #     to_show += "FNR: " + str(total_result['num_fn'] / total_result['num_detection']) + "\n"
    # to_show += "latency: " + str(np.average(total_result['delay'])) + " ms"
    # print(to_show)


    return results, results2, results3



In [ ]:
def save_json(result_list, name, seed, outdir="/content/drive/MyDrive/muia_experiment_results2022/storage_50"):
    path = os.path.join(outdir, str(seed))
    if os.path.exists(path) and os.path.isdir(path):
        print("Path exists, will overwrite the path")
    else:
        os.makedirs(path)

    with open(os.path.join(path, name + ".json"), 'w') as fp:
        json.dump(result_list, fp)

for i in range(2000, 5000, 100):

    results, results2, results3 = run_topic_robustness_type_cmp(i)
    save_json(results, 'distance', i)
    save_json(results2, 'random', i)
    save_json(results3, 'ideal', i)


========== SETUP ===========
Game name:robust100_2000_10_0_3
Negative trainer seed:100
Game seed:2000
trainers:815316, 776328, 261313, 326223, 501973, 879155, 389015, 525584, 240168, 675397
users:803262 (0), 962159 (1), 395129 (2)
testers:

Load data to memory. It may take up to 5 minutes...
Use Distance Storage
Player Set up:  auc_dempster_shafer True True
Use Random Storage
Player Set up:  auc_dempster_shafer True True
Use Unbounded Storage
Player Set up:  auc_dempster_shafer True True
User already registered.
User already registered.
803262 touch (248, 27) (27, 27)
962159 touch (47, 27) (5, 27)
touch
              precision    recall  f1-score   support

           0       1.00      0.60      0.75        10
           1       1.00      1.00      1.00        10
           2       0.56      1.00      0.71         5

    accuracy                           0.84        25
   macro avg       0.85      0.87      0.82        25
weighted avg       0.91      0.84      0.84        25

803262 g

/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:609: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/usr/local/lib/python3.7/dist-packages/scipy/interpolate/interpolate.py:612: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


gait
              precision    recall  f1-score   support

         0.0       0.97      0.90      0.93       100
         1.0       0.99      1.00      0.99        86
         2.0       0.92      0.97      0.94       100

    accuracy                           0.95       286
   macro avg       0.96      0.96      0.96       286
weighted avg       0.96      0.95      0.95       286

construct touch 50
construct touch 50
construct gait 500
construct gait 500
update storage size finished
gen 538363
986737 touch (45, 27) (5, 27)
990622 touch (45, 27) (5, 27)
538363 touch (98, 27) (10, 27)
touch
              precision    recall  f1-score   support

           0       0.71      0.50      0.59        10
           1       0.83      1.00      0.91         5
           2       1.00      1.00      1.00         5
           3       0.58      0.70      0.64        10

    accuracy                           0.73        30
   macro avg       0.78      0.80      0.78        30
weighted avg       0.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: 

gait
              precision    recall  f1-score   support

         0.0       0.86      0.80      0.83       104
         1.0       0.99      0.99      0.99       104
         2.0       0.98      0.94      0.96       104
         3.0       0.84      0.93      0.89       104

    accuracy                           0.92       416
   macro avg       0.92      0.92      0.92       416
weighted avg       0.92      0.92      0.92       416

construct touch 50
construct touch 50
construct touch 50
construct gait 500
construct gait 500
construct gait 500
update storage size finished
cached 506 features
cached 18 features
simulator says: falsely rejected me!
simulator says: falsely rejected me!
simulator says: falsely rejected me!
cached 226 features
cached 13 features
cached 339 features
cached 4 features
Not found
======= Player 2 Start ========
cached 12 features
cached 357 features
simulator says: falsely rejected me!
simulator says: falsely rejected me!
simulator says: falsely rejected me

...

gait
              precision    recall  f1-score   support

         0.0       0.28      0.07      0.11       100
         1.0       0.43      0.62      0.51        50
         2.0       0.39      0.84      0.54        50
         3.0       0.00      0.00      0.00         4

    accuracy                           0.39       204
   macro avg       0.28      0.38      0.29       204
weighted avg       0.34      0.39      0.31       204

construct touch 50
update storage size finished
cached 553 features
cached 19 features
[test] system capture user change 890725 890725 856302 763813
simulator says: I hear about user change
simulator says: ... but its wrong! 856302
system gets feedback, DENIED! 856302
too short, discarded
[test] system capture user change 965064 965064 856302 763813
simulator says: I hear about user change
simulator says: ... but its wrong! 856302
system gets feedback, DENIED! 856302
cached 506 features
cached 59 features
[test] system capture user change 1106018 1106018

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: 

gait
              precision    recall  f1-score   support

         0.0       0.60      0.47      0.53       100
         1.0       0.73      0.64      0.68        50
         2.0       0.41      0.68      0.52        50
         3.0       0.00      0.00      0.00         4

    accuracy                           0.55       204
   macro avg       0.44      0.45      0.43       204
weighted avg       0.58      0.55      0.55       204

construct touch 50
update storage size finished
cached 553 features
cached 19 features
cached 548 features
cached 62 features
[test] system capture user change 1111620 1111620 277905 763813
simulator says: I hear about user change
simulator says: ... but its wrong! 277905
system gets feedback, DENIED! 277905
too short, discarded
[test] system capture user change 1126006 1126006 277905 856302
simulator says: I hear about user change
simulator says: ... but its wrong! 277905
system gets feedback, DENIED! 277905
too short, discarded
too short, discarded
sim

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: 

gait
              precision    recall  f1-score   support

         0.0       0.34      0.12      0.18       100
         1.0       0.57      0.83      0.68       100
         2.0       0.52      0.65      0.58       100
         3.0       0.00      0.00      0.00         4

    accuracy                           0.53       304
   macro avg       0.36      0.40      0.36       304
weighted avg       0.47      0.53      0.47       304

update storage size finished
cached 553 features
cached 19 features
cached 548 features
cached 62 features
[test] system capture user change 1106018 1106018 277905 763813
simulator says: I hear about user change
simulator says: ... but its wrong! 277905
system gets feedback, DENIED! 277905
[test] system capture user change 1126006 1126006 277905 856302
simulator says: I hear about user change
simulator says: ... but its wrong! 277905
system gets feedback, DENIED! 277905
too short, discarded
too short, discarded
[test] system capture user change 1254719 1

In [ ]:
## Parsing Simulation Experiment Results
import json

def load_json_result(method, seed, base_dir):
    path = os.path.join(base_dir, str(seed), method + ".json")
    with open(path, 'r') as fp:
        return json.load(fp)

def load_all_json_results(seeds, base_dir):
    return {seed: {
        'distance': load_json_result('distance', seed, base_dir),
        'random': load_json_result('random', seed, base_dir),
        'unlimited': load_json_result('ideal', seed, base_dir),
    } for seed in seeds}

results = load_all_json_results(list(range(0,5000, 100)), "/content/drive/MyDrive/muia_experiment_results2022/storage_50")


In [ ]:
# parsing results


def parse_json_results(results):
    session_map = [(0, 0), (1, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]

    scoreboard = {
        i: [
         {'FR': [0, 0, 0], 'FI': [0, 0, 0]}
         for j in range(0, 3)
        ]
        for i in ['distance', 'random', 'unlimited']
    }
    for seed, result_board in results.items():
        for method, res in result_board.items():
            for i, r in enumerate(res):
                pos_a, pos_b = session_map[i]
                if r['num_fn'] > 0:
                    scoreboard[method][pos_a]['FR'][pos_b] += 1
                elif r['num_mc'] > 0:
                    scoreboard[method][pos_a]['FI'][pos_b] += 1
    return scoreboard

sb = parse_json_results(results)

In [ ]:
for key1, item1 in sb.items():
    for i in item1:
        for key2, item2 in i.items():
            print(key1, key2, np.array(item2)/50)

distance FR [0.2  0.28 0.24]
distance FI [0.02 0.04 0.04]
distance FR [0.38 0.3  0.24]
distance FI [0.06 0.06 0.  ]
distance FR [0.   0.3  0.22]
distance FI [0.   0.08 0.02]
random FR [0.22 0.3  0.26]
random FI [0.04 0.06 0.06]
random FR [0.34 0.22 0.18]
random FI [0.04 0.04 0.  ]
random FR [0.   0.32 0.08]
random FI [0.   0.04 0.04]
unlimited FR [0.14 0.34 0.22]
unlimited FI [0.02 0.02 0.04]
unlimited FR [0.36 0.14 0.2 ]
unlimited FI [0.04 0.   0.02]
unlimited FR [0.   0.32 0.16]
unlimited FI [0.   0.08 0.06]


In [ ]:


distance.cdist([[1,2,3]], [[4,5,6]])

array([[5.19615242]])